In [1]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import pandas as pd
import time
from datetime import date

# Replace these values with your actual API credentials
client_id = "PP860PBEU0-100"
secret_key = "JC501DPXH7"
redirect_uri = "https://google.com"
response_type = "code"  
state = "sample_state"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)
response = session.generate_authcode()
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=PP860PBEU0-100&redirect_uri=https%3A%2F%2Fgoogle.com&response_type=code&state=None


In [2]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTk2OTk1NjIsImV4cCI6MTY5OTcyOTU2MiwibmJmIjoxNjk5Njk4OTYyLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzk4MzY4Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiYmIyM2YxMjg2ODFiNGE3OWM2NmU2MTM4MDdkNjZhNGM5NmE5MjUzNDZhYjhiZWRkYzQ0MDdmMzYiLCJub25jZSI6IiIsImFwcF9pZCI6IlBQODYwUEJFVTAiLCJ1dWlkIjoiMjYwZjk4MTgxOTlmNDg5Y2JjMDM5ZTkzMzI5MjUwNjQiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.f8kCfhG1Ofo1iHQVmZq1AFxhd5ERnqSgUv1fJRIyzDY"

In [3]:
grant_type = "authorization_code"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
print(response)

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTk2OTk2NDEsImV4cCI6MTY5OTc0OTAyMSwibmJmIjoxNjk5Njk5NjQxLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbFQxdTVFTURScmY5YXVpTzltc3Q3UEYwXzNqWDM3bzlFOFRzR2poanJCcmVjUzVnZjhDZUdyUjZNbTF5YTg5X1BycjVsYmFuRk9qY29lUkVzN2x3eE80MDZmSWEwRnEyOUlHbzVpSHU0VVoxQkI1Zz0iLCJkaXNwbGF5X25hbWUiOiJTQU5TS0FSIFNSSVZBU1RBVkEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiYjIzZjEyODY4MWI0YTc5YzY2ZTYxMzgwN2Q2NmE0Yzk2YTkyNTM0NmFiOGJlZGRjNDQwN2YzNiIsImZ5X2lkIjoiWFM5ODM2OCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.vcB0k58LR_0JDyCZ7fiyscUeAjVGv1E8aWuv2R8Yt3s', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTk2OTk2NDEsImV4cCI6MTcwMDk1ODYyMSwibmJmIjoxNjk5Njk5NjQxLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQ

In [4]:
df=pd.DataFrame()
df=response
tk=df['access_token']
print(tk)

eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTk2OTk2NDEsImV4cCI6MTY5OTc0OTAyMSwibmJmIjoxNjk5Njk5NjQxLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbFQxdTVFTURScmY5YXVpTzltc3Q3UEYwXzNqWDM3bzlFOFRzR2poanJCcmVjUzVnZjhDZUdyUjZNbTF5YTg5X1BycjVsYmFuRk9qY29lUkVzN2x3eE80MDZmSWEwRnEyOUlHbzVpSHU0VVoxQkI1Zz0iLCJkaXNwbGF5X25hbWUiOiJTQU5TS0FSIFNSSVZBU1RBVkEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiYjIzZjEyODY4MWI0YTc5YzY2ZTYxMzgwN2Q2NmE0Yzk2YTkyNTM0NmFiOGJlZGRjNDQwN2YzNiIsImZ5X2lkIjoiWFM5ODM2OCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.vcB0k58LR_0JDyCZ7fiyscUeAjVGv1E8aWuv2R8Yt3s


In [5]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=tk, log_path="")

In [6]:
def signal_generator(df):
    Open = df.open.iloc[-1]
    Close = df.close.iloc[-1]
    previous_Open = df.open.iloc[-2]
    previous_Close = df.close.iloc[-2]
    
    # red candle
    if (Open>Close):
        return 1

    # green candle
    elif (Open<Close):
        return 2
    
    # No clear pattern
    else:
        return 0


In [24]:
call=input(str("Enter the symbol for call :"))
put=input(str("Enter the symbol for put :"))
calltight=int(input("Enter qty for tight lots for call option:  "))
callspare=int(input("Enter qty for spare lots for call option:  "))
puttight=int(input("Enter qty for tight lots for put option:  "))
putspare=int(input("Enter qty for spare lots for put option:  "))

Enter the symbol for call :NSE:BANKNIFTY23NOV43100CE
Enter the symbol for put :NSE:BANKNIFTY23NOV43700PE
Enter qty for tight lots for call option:  15
Enter qty for spare lots for call option:  15
Enter qty for tight lots for put option:  15
Enter qty for spare lots for put option:  15


In [25]:
date=date.today()
print(date)

2023-11-11


In [26]:
def get_datacall():
    data = {
    "symbol":call,
    "resolution":"5",
    "date_format":"1",
    "range_from":"2023-10-08",
    "range_to":"2023-11-10",
    "cont_flag":"1"}

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    #signal_generator(data)
    df["signal"] = signal
    
    goodpair = pd.DataFrame(columns=['Date', 'Time', 'HC', 'LC', 'ET', 'ES'])
    
    for i in range(1,len(df)):
        h=0
        l=0
        t=0
        signalfirst = df.signal.iloc[i]
        signalsecond = df.signal.iloc[i-1]
        presenthigh = df.high.iloc[i]
        previoushigh = df.high.iloc[i-1]
        presentlow = df.low.iloc[i]
        previouslow = df.low.iloc[i-1]
        previoustime = df.time.iloc[i-1]
        presenttime = df.time.iloc[i]
        previousdate = df.date.iloc[i-1]
        presentdate = df.date.iloc[i]
    
        if(signalfirst!=signalsecond):
            if(previoushigh>presenthigh and presentlow<previouslow):
                h = previoushigh
                l = presentlow
                t=1
            elif(previoushigh<presenthigh and presentlow>previouslow):
                h = presenthigh
                l = previouslow
                t=1
            elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=1):
                h=presenthigh
                l=presentlow
                t=1
            elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=1):
                h=previoushigh
                l=previouslow
                t=1
        if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
            goodpair.loc[len(goodpair)] = [previousdate, previoustime, h, l, 0, 0]
    global goodpairCE
    goodpairCE=goodpair

In [27]:
get_datacall()
goodpairCE

,Date,Time,HC,LC,ET,ES
0,2023-10-23,09:20:00,1369.10,1368.25,0,0
1,2023-10-25,09:35:00,1028.90,1012.80,0,0
2,2023-10-25,10:25:00,1020.00,1012.50,0,0
3,2023-10-25,12:05:00,925.00,899.00,0,0
4,2023-10-25,12:55:00,790.05,762.05,0,0
...,...,...,...,...,...,...
174,2023-11-10,09:55:00,855.40,850.00,0,0
175,2023-11-10,12:50:00,1042.25,1020.50,0,0
176,2023-11-10,13:10:00,1027.35,1009.15,0,0
177,2023-11-10,14:35:00,937.00,922.65,0,0


In [28]:
def get_dataput():
    data = {
    "symbol":put,
    "resolution":"5",
    "date_format":"1",
    "range_from":"2023-10-07",
    "range_to":"2023-11-11",
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    #signal_generator(data)
    df["signal"] = signal
    
    goodpair = pd.DataFrame(columns=['Date', 'Time', 'HP', 'LP', 'ET', 'ES'])
    
    for i in range(1,len(df)):
        h=0
        l=0
        t=0
        signalfirst = df.signal.iloc[i]
        signalsecond = df.signal.iloc[i-1]
        presenthigh = df.high.iloc[i]
        previoushigh = df.high.iloc[i-1]
        presentlow = df.low.iloc[i]
        previouslow = df.low.iloc[i-1]
        previoustime = df.time.iloc[i-1]
        presenttime = df.time.iloc[i]
        previousdate = df.date.iloc[i-1]
        presentdate = df.date.iloc[i]
    
        if(signalfirst!=signalsecond):
            if(previoushigh>presenthigh and presentlow<previouslow):
                h = previoushigh
                l = presentlow
                t=1
            elif(previoushigh<presenthigh and presentlow>previouslow):
                h = presenthigh
                l = previouslow
                t=1
            elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=1):
                h=presenthigh
                l=presentlow
                t=1
            elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=1):
                h=previoushigh
                l=previouslow
                t=1
        if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
            goodpair.loc[len(goodpair)] = [previousdate, previoustime, h, l, 0, 0]
    global goodpairPE
    goodpairPE=goodpair

In [29]:
get_dataput()
goodpairPE

,Date,Time,HP,LP,ET,ES
0,2023-10-10,14:50:00,412.60,407.80,0,0
1,2023-10-11,10:50:00,402.35,389.00,0,0
2,2023-10-12,10:50:00,342.00,334.00,0,0
3,2023-10-17,09:20:00,336.25,330.25,0,0
4,2023-10-17,09:30:00,344.15,335.65,0,0
...,...,...,...,...,...,...
218,2023-11-10,13:40:00,360.90,354.20,0,0
219,2023-11-10,13:45:00,364.00,354.20,0,0
220,2023-11-10,13:55:00,380.65,360.90,0,0
221,2023-11-10,14:05:00,369.10,357.80,0,0


In [31]:
def onmessage(message):
    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)
    if (int(cmin) % 5 == 0 and int(csec)<3):
        get_datacall()
        get_dataput()
        '''if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today()):
            global hc
            hc = goodpairCE.HC.iloc[-1]
            global lc
            lc = goodpairCE.LC.iloc[-1]
        if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today()):
            global hp
            hp = goodpairPE.HP.iloc[-1]
            global lp
            lp = goodpairPE.LP.iloc[-1]'''
    print("Response : ", message)
    if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ET.iloc[-1]==0):
        data_ct = {
            "symbol":call,
            "qty":calltight,
            "type":1,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.LC.iloc[-1],
            "takeProfit":goodpairCE.HC.iloc[-1]+15
        }
        response_ct = fyers.place_order(data=data_ct)
        #print(response_ct)
        if response_ct['s']=='ok':
            print(f"entry tight slot for call {call}")
            goodpairCE.ET.iloc[-1]=1
        
    if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ET.iloc[-1]==0):
        data_pt = {
            "symbol":put,
            "qty":puttight,
            "type":1,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.LP.iloc[-1],
            "takeProfit":goodpairPE.HP.iloc[-1]+15
        }
        response_pt = fyers.place_order(data=data_pt)
        if response_pt['s']=='ok':
            print(f"entry tight slot for put {put}")
            goodpairPE.ET.iloc[-1]=1
            
    if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ES.iloc[-1]==0):
        data_cs = {
            "symbol":call,
            "qty":callspare,
            "type":1,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.LC.iloc[-1],
            "takeProfit":0
        }
        global response_cs
        response_cs = fyers.place_order(data=data_cs)
        if response_cs['s']=='ok':
            print(f"entry spare slot for call {call}")
            global targetC
            targetC=goodpairCE.HC.iloc[-1]+15
            global slC
            slC=goodpairCE.LC.iloc[-1]
            goodpairCE.ES.iloc[-1]=1
            
    if(goodpairCE.ES.iloc[-1]==1 and message['symbol']==call and message['ltp']>=targetC):
        targetC=targetC+15
        slC=targetC-30
        orderIdC = response_cs['id']
        data_mcs = {
            "id":orderIdC, 
            "type":1, 
            "limitPrice":slC,
            "qty":callspare
        }
        response_mcs = fyers.modify_order(data=data_mcs)
        if response_mcs['s']=='ok':
            print(f"spare slot trailed for call {call} at sl={slC}")
            
    if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ES.iloc[-1]==0):
        data_ps = {
            "symbol":put,
            "qty":putspare,
            "type":1,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.LP.iloc[-1],
            "takeProfit":0
        }
        global response_ps
        response_ps = fyers.place_order(data=data_ps)
        if response_ps['s']=='ok':
            print(f"entry spare slot for put {put}")
            global targetP
            targetP=goodpairPE.HP.iloc[-1]+15
            global slP
            slP=goodpairPE.LP.iloc[-1]
            goodpairPE.ES.iloc[-1]=1
            
    if(goodpairPE.ES.iloc[-1]==1 and message['symbol']==put and message['ltp']>=targetP):
        targetP=targetP+15
        slP=targetP-30
        orderIdP = response_ps['id']
        data_mps = {
            "id":orderIdP, 
            "type":1, 
            "limitPrice": slP,
            "qty":putspare
        }
        response_mps = fyers.modify_order(data=data_mps)
        if response_mps['s']=='ok':
            print(f"spare slot trailed for put {put} at sl={slP}")

def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"
    symbols = [call,put]
    #print(len(goodpairPE))
    ofyers.subscribe(symbols=symbols, data_type=data_type)
    ofyers.keep_running()
access_token = client_id + ":" + tk

ofyers = data_ws.FyersDataSocket(
    access_token=access_token,
    log_path="",                     
    litemode=False,
    write_to_file=False,              
    reconnect=True,                 
    on_connect=onopen,               
    on_close=onclose,                
    on_error=onerror,               
    on_message=onmessage             
)
ofyers.connect()

Response :  {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Response :  {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Response :  {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Response :  {'ltp': 0.0, 'vol_traded_today': 0, 'exch_feed_time': 1696327201, 'bid_size': 810, 'ask_size': 135, 'bid_price': 413.8, 'ask_price': 536.0, 'last_traded_qty': 0, 'tot_buy_qty': 960, 'tot_sell_qty': 135, 'avg_trade_price': 0.0, 'low_price': 0.0, 'high_price': 0.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 0.0, 'prev_close_price': 1280.95, 'type': 'sf', 'symbol': 'NSE:BANKNIFTY23NOV43700PE', 'ch': -1280.95, 'chp': -100.0}
Response :  {'ltp': 0.0, 'vol_traded_today': 0, 'exch_feed_time': 1696327201, 'bid_size': 615, 'ask_size': 600, 'bid_price': 1378.6, 'ask_price': 2378.05, 'last_traded_qty': 0, 'tot_buy_qty': 630, 'tot_sell_qty': 3285, 'avg_trade_price': 0.0, 'low_price': 0.0, 'high_price': 0.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_p